In [1]:
from dotenv import load_dotenv

load_dotenv("../config/.env")

True

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

class Person(BaseModel):
    """关于一个人的信息。"""
    name: Optional[str] = Field(
        default=None, description="这个人的名字"
    )
    hair_color: Optional[str] = Field(
        default=None, description="这个人的头发颜色，如果有的话"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="以米为单位的身高"
    )

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个专业的提取算法。"
            "只从文本中提取相关信息。"
            "如果你不知道所要求提取的属性的值，"
            "则返回该属性值为 null。”",
        ),
        ("human", "{text}"),
    ]
)

text = "Frank 身高 6 英尺，有着一头黑发。"
prompt = prompt_template.invoke({"text": text})

llm = init_chat_model("deepseek-chat", model_provider="deepseek")
structured_llm = llm.with_structured_output(schema=Person)

structured_llm.invoke(prompt)

Person(name='Frank', hair_color='黑发', height_in_meters=None)

In [6]:
from typing import List

class Data(BaseModel):
    people: List[Person]

multiple_entities_text = "我叫Frank，我的头发是黑色的，身高 6 英尺。Daneila的头发颜色和我一样。"
multiple_entities_prompt = prompt_template.invoke(multiple_entities_text)
multiple_entities_structured_llm = llm.with_structured_output(schema=Data)
multiple_entities_structured_llm.invoke(multiple_entities_prompt)

Data(people=[Person(name='Frank', hair_color='黑色', height_in_meters=None), Person(name='Daneila', hair_color='黑色', height_in_meters=None)])